In [2]:
import csv
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.tokenize import regexp_tokenize

#  Create documents

In [ ]:
with open('Airbnb_Texas_Rentals.tsv', 'r', encoding = "utf8") as all_rev:
    csv_reader = csv.reader(all_rev)
    ind = -1
    for row in all_rev:
        ind += 1
        if ind == 0:
            continue
        else:
            if row == '\n':
                ind -= 1
            else:
                new_title = r'C:\Users\Daniele\fileAIR\doc_' + str(ind) + '.tsv'
                csv_writer = csv.writer(open(new_title, 'w', encoding = "utf8"), delimiter ='\t')
                row1 = row.split("\t")[1:]
                csv_writer.writerow(row1)

# Search Engine

In [3]:
def clean1(something):
    something=something.lower()
    something=something.replace('\\n',' ')
    tokenizer = regexp_tokenize(something,"[\w'\$]+")
    filtered = [w for w in tokenizer if not w in stopwords.words('english')]
    ps=PorterStemmer()
    filtered=[ps.stem(word) for word in filtered]
    return filtered

In [4]:
with open('doc_1.tsv', 'r') as csvfile:
    file1 = csv.reader(csvfile, delimiter='\t')
    columns = [i for i in file1]
    description = columns[0][4]
    description=clean1(description)

In [26]:
test={k:v for v, k in enumerate(description, 1)}

In [27]:
test.keys()

dict_keys(['welcom', 'stay', 'privat', 'room', 'queen', 'bed', 'detach', 'bathroom', 'second', 'floor', 'anoth', 'bedroom', 'sofa', 'avail', 'addit', 'guest', '10$', '10min', 'iah', 'airport', 'pick', 'drop', '$10', 'trip'])

In [30]:
for key in test.keys():
    print(test[key])

1
2
13
4
5
16
7
9
10
11
12
14
15
29
21
22
20
23
24
26
27
28
30
31


In [38]:
test

{'$10': 30,
 '10$': 20,
 '10min': 23,
 'addit': 21,
 'airport': 26,
 'anoth': 12,
 'avail': 29,
 'bathroom': 9,
 'bed': 16,
 'bedroom': 14,
 'detach': 7,
 'drop': 28,
 'floor': 11,
 'guest': 22,
 'iah': 24,
 'pick': 27,
 'privat': 13,
 'queen': 5,
 'room': 4,
 'second': 10,
 'sofa': 15,
 'stay': 2,
 'trip': 31,
 'welcom': 1}

In [5]:
import os,os.path

In [3]:
print(len([x for x in os.scandir(r'C:\Users\Daniele\airbnb')]))

264


In [45]:
vocabulary_set=set()
docs_list=[]

In [64]:
for i in range(1,len([x for x in os.scandir(r'C:\Users\Daniele\airbnb')])):
    with open(r'C:\Users\Daniele\airbnb\doc_'+str(i)+'.tsv', 'r',encoding='utf8') as csvfile:
        file1 = csv.reader(csvfile, delimiter='\t')
        columns = [i for i in file1]
        description = columns[0][4]
        description=clean1(description)
        vocabulary_set.update(description)
        docs_list.append(set(description))

In [74]:
vocabulary = {k:v for v, k in enumerate(vocabulary_set)}

In [80]:
# Save vocabulary to the file
voc_file = open(r'C:\Users\Daniele\vocabulary.txt','w',encoding="utf8")
for term in vocabulary:
    voc_file.write('{0}\t{1}\n'.format(term, vocabulary[term]))

In [66]:
from collections import defaultdict #we use defaultdict because it gives no key error
#we need t sort this value
inv_indx = defaultdict(set)
for idx, text in enumerate(docs_list):
    for word in text:
        id_word = vocabulary[word]
        inv_indx[id_word].add(idx)


In [72]:
for key in inv_indx:
    value = inv_indx[key]
    inv_indx[key] = list(value)

In [73]:
inv_indx

defaultdict(set,
            {916: [0],
             556: [0, 54],
             1320: [0, 170, 47, 175, 210, 52, 53, 85, 63],
             1527: [0, 134, 55],
             1633: [0, 108, 92],
             1576: [0,
              1,
              129,
              3,
              258,
              259,
              134,
              7,
              8,
              10,
              138,
              142,
              16,
              145,
              18,
              20,
              21,
              150,
              151,
              25,
              153,
              159,
              160,
              33,
              163,
              164,
              168,
              169,
              44,
              172,
              173,
              47,
              48,
              175,
              176,
              178,
              52,
              179,
              182,
              55,
              184,
              57,
              58,
         

In [87]:
# Save inv_indx to the file
inv_file = open(r'C:\Users\Daniele\inverted_indx.txt','w',encoding="utf8")
for id_term in inv_indx:
    docks = inv_indx[id_term]
    d = '\t'.join(map(str, docks))
    inv_file.write('{0}\t{1}\n'.format(id_term, d))
    